In [3]:
#Libraries to use
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#Machine Learning
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

Supervised Learning: model is getting trained on a labelled dataset.

Classification model: ​Predict the category the input data belong to​, classify new sample into the existing groups.
    Binary Classification: goal is to classify the input into one of two classes or categories

Ideal outcome: Determine if a fire is caused by a human or lighting.
Model's goal: Predict if the cause of a fire is human activity or lighting activity.
Model's output: Binary classification (Human or lighting)

Success metrics: The model classified the cause of the fire. 

Feature Selection method:
IF Input: categorical AND Output: categorical we use Chi-square test (contigency tables) OR mutual information.

Possible ML classification models:
* Logistic Regression​
type of classification algorithm that is used to predict a binary output variable​
commonly used in machine learning applications where the output variable is either true or false, such as in fraud detection or spam filtering​
the algorithm tries to find a linear relationship between the input features and the output variable​
The output variable is then transformed using a logistic function to produce a probability value between 0 and 1.

* Decision Trees​
type of algorithm that is used for both classification and regression tasks​
tree-like structure that is used to model decisions and their possible consequences.​
Each internal node in the tree represents a decision, while each leaf node represents a possible outcome​
Decision trees can be used to model complex relationships between input features and output variables.

* Random Forests​
ensemble learning technique that is used for both classification and regression tasks​
made up of multiple decision trees that work together to make predictions​
Each tree in the forest is trained on a different subset of the input features and data​
The final prediction is made by aggregating the predictions of all the trees in the forest

Evaluation metrics:
* Accuaracy
* hold out
* cross validation
* bootstraping
* confusion matrix

In [4]:
Firedf = pd.read_csv("./FIREANAL_V2.csv")
Firedf.head()

,Serial,FireYear,Area,DistrictName,UnitName,FireName,Size_class,EstTotalAcres,Protected_Acres,HumanOrLightning,...,Industrial_Restriction,Ign_DateTime,ReportDateTime,Discover_DateTime,Control_DateTime,CreationDate,ModifiedDate,DistrictCode,UnitCode,DistFireNumber
0,58256,2000,EOA,Central Oregon,John Day,Slick Ear #2,B,0.75,0.75,Lightning,...,Does Not Apply - Eastern OR,07/18/2000 07:00:00 PM,07/19/2000 01:20:00 PM,07/19/2000 01:15:00 PM,07/20/2000 12:50:00 AM,07/20/2000 09:13:00 AM,11/14/2000 09:16:00 AM,95,952,11
1,59312,2000,EOA,Northeast Oregon,La Grande,Woodley,C,80.00,80.00,Lightning,...,Does Not Apply - Eastern OR,08/24/2000 05:30:00 AM,08/24/2000 01:07:00 PM,08/24/2000 01:07:00 PM,09/01/2000 21:30,08/29/2000 03:59:00 PM,12/21/2000 04:22:00 PM,97,971,24
2,61657,2001,SOA,Southwest Oregon,Grants Pass,QUEENS BRANCH,A,0.10,0.10,Human,...,Lvl 3 Restricted Shutdown,08/10/2001 17:40,08/10/2001 17:47,08/10/2001 17:45,08/10/2001 18:30,08/10/2001 18:42,08/17/2001 11:45:00 AM,71,712,133
3,63735,2002,NOA,West Oregon,Philomath,WREN,A,0.01,0.01,Human,...,Lvl 1 Fire Season Only,07/06/2002 13:01,07/06/2002 13:04,07/06/2002 13:02,07/06/2002 13:07,07/07/2002 09:16,07/28/2002 10:08:00 AM,55,551,1
4,68019,2003,NOA,West Oregon,Dallas,Ritner Creek,A,0.01,0.01,Lightning,...,Lvl 3 Restricted Shutdown,08/22/2003 04:00:00 AM,08/22/2003 05:00:00 AM,08/22/2003 05:00:00 AM,08/22/2003 09:30:00 AM,08/22/2003 04:12:00 PM,09/04/2003 09:39,55,552,13


In [5]:
c = Firedf.drop(["FireYear", "FireName",
 "Size_class","EstTotalAcres", "Protected_Acres", "Sec","RegUseRestriction",
  "Industrial_Restriction", "Ign_DateTime", "ReportDateTime", "Discover_DateTime", "Control_DateTime", "CreationDate",
  "ModifiedDate", "DistFireNumber"], axis=1, inplace=True)

In [6]:
Firedf.head()

,Serial,Area,DistrictName,UnitName,HumanOrLightning,Lat_DD,Long_DD,FO_LandOwnType,County,DistrictCode,UnitCode
0,58256,EOA,Central Oregon,John Day,Lightning,44.91519,-119.28863,BLM,Grant,95,952
1,59312,EOA,Northeast Oregon,La Grande,Lightning,45.08509,-118.33440,Other Private,Union,97,971
2,61657,SOA,Southwest Oregon,Grants Pass,Human,42.53671,-123.21215,BLM,Jackson,71,712
3,63735,NOA,West Oregon,Philomath,Human,44.58709,-123.42779,State,Benton,55,551
4,68019,NOA,West Oregon,Dallas,Lightning,44.74026,-123.49811,Industrial,Polk,55,552


In [7]:
#We have check that ther are 7 rows how have "under investment" as HumanOrLightning, so we are deleting this for better calculations
# Define which string we want to delete
string_to_del = 'Under Invest'

# Here we delete that string from all the rows
Firedf = Firedf[Firedf['HumanOrLightning'] != string_to_del]

# we check
Firedf

,Serial,Area,DistrictName,UnitName,HumanOrLightning,Lat_DD,Long_DD,FO_LandOwnType,County,DistrictCode,UnitCode
0,58256,EOA,Central Oregon,John Day,Lightning,44.91519,-119.28863,BLM,Grant,95,952
1,59312,EOA,Northeast Oregon,La Grande,Lightning,45.08509,-118.33440,Other Private,Union,97,971
2,61657,SOA,Southwest Oregon,Grants Pass,Human,42.53671,-123.21215,BLM,Jackson,71,712
3,63735,NOA,West Oregon,Philomath,Human,44.58709,-123.42779,State,Benton,55,551
4,68019,NOA,West Oregon,Dallas,Lightning,44.74026,-123.49811,Industrial,Polk,55,552
...,...,...,...,...,...,...,...,...,...,...,...
23485,124548,EOA,Walker Range - WRFPA,Crescent,Human,43.52875,-121.35672,Industrial,Klamath,99,991
23486,132141,SOA,Western Lane,Veneta,Human,43.97600,-124.09933,Municipal,Lane,78,781
23487,124065,EOA,Northeast Oregon,Pendleton,Lightning,45.01056,-119.08639,BLM,Umatilla,97,973
23488,131292,EOA,Northeast Oregon,Pendleton,Human,45.58639,-118.46167,State,Umatilla,97,973


In [8]:
Firedf.count()

Serial              23463
Area                23463
DistrictName        23463
UnitName            23463
HumanOrLightning    23463
Lat_DD              23455
Long_DD             23455
FO_LandOwnType      23463
County              23463
DistrictCode        23463
UnitCode            23463
dtype: int64

In [23]:
Firedf['HumanMade'] = Firedf['HumanOrLightning'].apply(lambda x: 0 if x == 'Lightning' else 1)
Firedf.head()

,Serial,Area,DistrictName,UnitName,HumanOrLightning,Lat_DD,Long_DD,FO_LandOwnType,County,DistrictCode,UnitCode,HumanMade
0,58256,EOA,Central Oregon,John Day,Lightning,44.91519,-119.28863,BLM,Grant,95,952,0
1,59312,EOA,Northeast Oregon,La Grande,Lightning,45.08509,-118.33440,Other Private,Union,97,971,0
2,61657,SOA,Southwest Oregon,Grants Pass,Human,42.53671,-123.21215,BLM,Jackson,71,712,1
3,63735,NOA,West Oregon,Philomath,Human,44.58709,-123.42779,State,Benton,55,551,1
4,68019,NOA,West Oregon,Dallas,Lightning,44.74026,-123.49811,Industrial,Polk,55,552,0


In [24]:
SFiredf = Firedf
SFiredf.head()

,Serial,Area,DistrictName,UnitName,HumanOrLightning,Lat_DD,Long_DD,FO_LandOwnType,County,DistrictCode,UnitCode,HumanMade
0,58256,EOA,Central Oregon,John Day,Lightning,44.91519,-119.28863,BLM,Grant,95,952,0
1,59312,EOA,Northeast Oregon,La Grande,Lightning,45.08509,-118.33440,Other Private,Union,97,971,0
2,61657,SOA,Southwest Oregon,Grants Pass,Human,42.53671,-123.21215,BLM,Jackson,71,712,1
3,63735,NOA,West Oregon,Philomath,Human,44.58709,-123.42779,State,Benton,55,551,1
4,68019,NOA,West Oregon,Dallas,Lightning,44.74026,-123.49811,Industrial,Polk,55,552,0


In [25]:
SFiredf.describe(include="all")

,Serial,Area,DistrictName,UnitName,HumanOrLightning,Lat_DD,Long_DD,FO_LandOwnType,County,DistrictCode,UnitCode,HumanMade
count,23463.000000,23463,23463,23463,23463,23455.000000,23455.000000,23463,23463,23463.000000,23463.000000,23463.000000
unique,NaN,3,14,31,2,NaN,NaN,15,37,NaN,NaN,NaN
top,NaN,SOA,Southwest Oregon,Grants Pass,Human,NaN,NaN,Rural Residential,Jackson,NaN,NaN,NaN
freq,NaN,12119,5975,3072,17197,NaN,NaN,5786,3338,NaN,NaN,NaN
mean,90398.315603,NaN,NaN,NaN,NaN,43.718386,-122.259300,NaN,NaN,77.751907,779.343477,0.732941
std,20892.909846,NaN,NaN,NaN,NaN,1.226331,1.707741,NaN,NaN,14.613757,146.483946,0.442433
min,57485.000000,NaN,NaN,NaN,NaN,41.089460,-124.983610,NaN,NaN,51.000000,511.000000,0.000000
25%,72498.500000,NaN,NaN,NaN,NaN,42.526115,-123.387765,NaN,NaN,71.000000,711.000000,0.000000
50%,88294.000000,NaN,NaN,NaN,NaN,43.581670,-122.848330,NaN,NaN,73.000000,732.000000,1.000000
75%,107391.500000,NaN,NaN,NaN,NaN,44.789460,-121.543060,NaN,NaN,95.000000,954.000000,1.000000


In [26]:
#Check for Null Values, we have 8
null_counts = SFiredf.isnull().sum()
print(null_counts)

Serial              0
Area                0
DistrictName        0
UnitName            0
HumanOrLightning    0
Lat_DD              8
Long_DD             8
FO_LandOwnType      0
County              0
DistrictCode        0
UnitCode            0
HumanMade           0
dtype: int64


In [27]:
SFiredf.replace('', np.nan, inplace=True)

In [28]:
#Here we delete the null values (rows)
SFiredftest = SFiredf.dropna(how='any')

In [29]:
#Validation that the null values are deleted
null_counts = SFiredftest.isnull().sum()
print(null_counts)

Serial              0
Area                0
DistrictName        0
UnitName            0
HumanOrLightning    0
Lat_DD              0
Long_DD             0
FO_LandOwnType      0
County              0
DistrictCode        0
UnitCode            0
HumanMade           0
dtype: int64


In [30]:
#Spliting of data
train, test = train_test_split(SFiredftest, test_size=0.3, random_state=42)

In [31]:
train.shape[0] / SFiredftest.shape[0]

0.6999786825836709

In [32]:
train.head()

,Serial,Area,DistrictName,UnitName,HumanOrLightning,Lat_DD,Long_DD,FO_LandOwnType,County,DistrictCode,UnitCode,HumanMade
8140,81533,EOA,Central Oregon,Sisters,Lightning,44.33194,-121.73417,Small Woodlands,Deschutes,95,955,0
19422,91163,NOA,Astoria,Astoria,Human,45.86444,-123.59861,Other Private,Clatsop,52,521,1
4626,60573,NOA,Forest Grove,Columbia City,Human,45.95931,-122.95214,Rural Residential,Columbia,53,532,1
9854,86358,SOA,Southwest Oregon,Medford,Lightning,42.67556,-122.84944,BLM,Jackson,71,711,0
223,59968,SOA,Southwest Oregon,Medford,Human,42.16528,-122.80821,Industrial,Jackson,71,711,1


In [33]:
human = train.loc[train.HumanOrLightning == 'Human']["HumanMade"]
rate_human = len(human)/len(train)

print("% of human who made fire:", rate_human)

% of human who made fire: 0.7339505420879523


In [34]:
Lightning = train.loc[train.HumanOrLightning == 'Lightning']["HumanMade"]
rate_Lightning = len(Lightning)/len(train)

print("% of lightnings who made fire:", rate_Lightning)

% of lightnings who made fire: 0.26604945791204776


In [35]:
y = train["HumanMade"]

features = ["Area", "DistrictName","UnitName","Lat_DD","Long_DD","County","DistrictCode","UnitCode"]
X = pd.get_dummies(train[features])
X_test = pd.get_dummies(test[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'Serial': test.Serial, 'HumanMadePrediction': predictions, "HumanMadeReal": test.HumanMade})
output.to_csv('Fire_Predictions.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [36]:
y_pred = cross_val_predict(model, X, y, cv=5)

accuracy = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)
confusion_mat = confusion_matrix(y, y_pred)

# Print metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion_mat)

Accuracy: 0.7937020343525399
Precision: 0.8171633594493666
Recall: 0.9261410788381743
F1 Score: 0.868246003034193
Confusion Matrix:
[[ 1871  2497]
 [  890 11160]]
